In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# torch.manual_seed(0)

from torch.autograd import Variable

#from torchsummary import summary

import torch.optim as optim
import time
import matplotlib
from torch.utils.data import DataLoader
import torch.utils.data as data
import numpy as np
import pandas as pd
from torchvision.datasets import ImageFolder
from torchvision import transforms

import cv2

In [4]:
import sys,os
sys.path.append(os.pardir)

import py_func.Deep_learning_torch_func as DL
import py_func.models_func as my_model
import py_func.dat_rb_func as drb

In [5]:
from torchinfo import summary

In [31]:
simpath = '../dat/expedata/m_23001_25000_fps2_N10_lsd.dat'
binpath = '../dat/bindat/1byte/m_28_1.dat'
x = 28
y = 28

#復元
m_dc_set_t,m_dc_set_v=DL.mysimbin_to_dataset_ver2(simpath, binpath, 1600, x, y, 2000, 1, 1, True, 100,  normalize_true = False, label_img_norm_true = False)
m_dc_loader_t=DataLoader(m_dc_set_t,batch_size=64,shuffle=True)
m_dc_loader_v=DataLoader(m_dc_set_v,batch_size=64,shuffle=True)

In [32]:
criterion = nn.MSELoss()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#model_ver3
#28x28出力
kernel_size = [6,4,4,4,4,4]
stride_list = [1,1,1,1,1,1]
padding_list = [4,4,2,2,2,2]
outpadding_list = [0,0,0,0,0,0]
in_channel_list = [1,64,128,256,128,64]
out_channel_list = [64,128,256,128,64,1]

model = my_model.simnet_decoder_allsize(1600, 1600, 40, 40, kernel_size, stride_list, padding_list,
                                        outpadding_list, in_channel_list, out_channel_list).to(device)


In [33]:
summary(model = model, input_size = (64, 1, 1600))

Layer (type:depth-idx)                        Output Shape              Param #
simnet_decoder_allsize                        --                        --
├─Linear: 1-1                                 [64, 1, 1600]             2,561,600
├─ReLU: 1-2                                   [64, 1, 1600]             --
├─Sequential: 1-3                             [64, 1, 28, 28]           --
│    └─Convtp_Bn_ReLu: 2-1                    [64, 64, 37, 37]          --
│    │    └─Sequential: 3-1                   [64, 64, 37, 37]          2,496
│    └─Convtp_Bn_ReLu: 2-2                    [64, 128, 32, 32]         --
│    │    └─Sequential: 3-2                   [64, 128, 32, 32]         131,456
│    └─Convtp_Bn_ReLu: 2-3                    [64, 256, 31, 31]         --
│    │    └─Sequential: 3-3                   [64, 256, 31, 31]         525,056
│    └─Convtp_Bn_ReLu: 2-4                    [64, 128, 30, 30]         --
│    │    └─Sequential: 3-4                   [64, 128, 30, 30]         524